In [1]:
import pandas as pd
import plotly.express as px

### EDA

In [2]:
import pandas as pd

df = pd.read_csv(r'C:\Users\Mohak kansal\Documents\Coding\Projects\IPL Auction Analytics & Team Performance Prediction Dashboard\Data Collection and Cleaning\Used Datasets\3.) IPL AUCTION 2024 DATASET\IPL AUCTION 2024 DATASET.csv')
df.head()


,Unnamed: 0,Player,Base Price,TYPE,COST IN ₹ (CR.),Cost IN $ (000),2023 Squad,Team
0,0,Spencer Johnson,5000000,BOWLER,10.0,1200.0,NaN,Gujarat Titans
1,1,Shahrukh Khan,4000000,ALL-ROUNDER,7.4,888.0,PBKS,Gujarat Titans
2,2,Umesh Yadav,20000000,BOWLER,5.8,696.0,KKR,Gujarat Titans
3,3,Robin Minz,2000000,WICKETKEEPER,3.6,432.0,NaN,Gujarat Titans
4,4,Sushant Mishra,2000000,BOWLER,2.2,264.0,NaN,Gujarat Titans


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675 entries, 0 to 674
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       675 non-null    int64  
 1   Player           675 non-null    object 
 2   Base Price       675 non-null    object 
 3   TYPE             333 non-null    object 
 4   COST IN ₹ (CR.)  240 non-null    float64
 5   Cost IN $ (000)  240 non-null    float64
 6   2023 Squad       67 non-null     object 
 7   Team             675 non-null    object 
dtypes: float64(2), int64(1), object(5)
memory usage: 42.3+ KB


In [4]:
df.describe()

,Unnamed: 0,COST IN ₹ (CR.),Cost IN $ (000)
count,675.0,240.000000,240.000000
mean,337.0,0.960208,112.925000
std,195.0,2.934421,351.066842
min,0.0,0.000000,0.000000
25%,168.5,0.000000,0.000000
50%,337.0,0.000000,0.000000
75%,505.5,0.200000,24.000000
max,674.0,24.750000,2970.000000


In [5]:
df.isnull().sum()

Unnamed: 0           0
Player               0
Base Price           0
TYPE               342
COST IN ₹ (CR.)    435
Cost IN $ (000)    435
2023 Squad         608
Team                 0
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df.columns

Index(['Unnamed: 0', 'Player', 'Base Price', 'TYPE', 'COST IN ₹ (CR.)',
       'Cost IN $ (000)', '2023 Squad', 'Team'],
      dtype='object')

#### Unique Values in Key Columns

In [8]:
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

Unnamed: 0: 675 unique values
Player: 503 unique values
Base Price: 10 unique values
TYPE: 4 unique values
COST IN ₹ (CR.): 32 unique values
Cost IN $ (000): 31 unique values
2023 Squad: 10 unique values
Team: 11 unique values


#### Top 10 Most Expensive Buys

In [9]:
top_buys = df.sort_values('COST IN ₹ (CR.)', ascending=False).head(10)
fig = px.bar(top_buys, x='Player', y='COST IN ₹ (CR.)', color='Team', title='Top 10 Most Expensive Buys (2024 Auction)')
fig.show()

#### Total Spend by Each Team

In [10]:
team_spend = df.groupby('Team')['COST IN ₹ (CR.)'].sum().reset_index()
fig = px.bar(team_spend, x='Team', y='COST IN ₹ (CR.)', title='Total Spend by Team (2024 Auction)')
fig.show()

#### Role-wise Spend Distribution

In [11]:
role_spend = df.groupby('TYPE')['COST IN ₹ (CR.)'].sum().reset_index()
fig = px.pie(role_spend, names='TYPE', values='COST IN ₹ (CR.)', title='Role-wise Spend Distribution')
fig.show()

#### Number of Players Bought by Each Team

In [12]:
team_players = df['Team'].value_counts().reset_index()
team_players.columns = ['Team', 'Players Bought']
fig = px.bar(team_players, x='Team', y='Players Bought', title='Number of Players Bought by Each Team')
fig.show()

#### Base Price vs. Final Price Scatter Plot

In [13]:
# Filter out retained/traded players for this plot
auction_buys = df[df['Base Price'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
fig = px.scatter(auction_buys, x='Base Price', y='COST IN ₹ (CR.)', color='TYPE', hover_data=['Player', 'Team'],
                title='Base Price vs Final Price (Auction Buys Only)')
fig.show()

#### Top 10 Bargain Buys (Lowest Price for High Base Price)

In [14]:
auction_buys = df[df['Base Price'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
auction_buys['Bargain Index'] = auction_buys['Base Price'].astype(float) / auction_buys['COST IN ₹ (CR.)'].replace(0, 1)
top_bargains = auction_buys.sort_values('Bargain Index', ascending=False).head(10)
fig = px.bar(top_bargains, x='Player', y='Bargain Index', color='Team', title='Top 10 Bargain Buys (2024 Auction)')
fig.show()

C:\Users\Mohak kansal\AppData\Local\Temp\ipykernel_17656\4269992280.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Data Cleaning

#### Remove null values

In [15]:
df['COST IN ₹ (CR.)'] = df['COST IN ₹ (CR.)'].fillna(0)
df['Cost IN $ (000)'] = df['Cost IN $ (000)'].fillna(0)

df['2023 Squad'] = df['2023 Squad'].fillna("Uncapped")


#### Adjusting columns

In [16]:
df = df.drop(columns=['Unnamed: 0'])


In [17]:
df = df.rename(columns={'TYPE': 'Type', 'COST IN ₹ (CR.)': 'Cost IN ₹ (CR.)', 'Cost IN $ (000)': 'Cost IN $ (K)'})

In [18]:
print(df['Team'].unique())
print(df['Type'].unique())

['Gujarat Titans' 'Chennai Super Kings' 'Delhi Capital'
 'Kolkata Knight Riders' 'Punjab Kings' 'Lucknow Super Giants'
 'Mumbai Indians' 'Royal Challengers Bengaluru' 'Rajasthan Royals'
 'Sunrisers Hyderabad' 'Unsold']
['BOWLER' 'ALL-ROUNDER' 'WICKETKEEPER' nan 'BATTER']


In [19]:
df['Type'] = df['Type'].astype(str).str.strip().str.title()
print(df['Type'].unique())

['Bowler' 'All-Rounder' 'Wicketkeeper' 'Nan' 'Batter']


In [20]:
df.to_csv('IPL_AUCTION_2024_DATASET_CLEANED.csv', index=False)

In [21]:
df = pd.read_csv(r'C:\Users\Mohak kansal\Documents\Coding\Projects\IPL Auction Analytics & Team Performance Prediction Dashboard\Data Collection and Cleaning\Used Datasets\3.) IPL AUCTION 2024 DATASET\IPL_AUCTION_2024_DATASET_CLEANED.csv')
df.head()

,Player,Base Price,Type,Cost IN ₹ (CR.),Cost IN $ (K),2023 Squad,Team
0,Spencer Johnson,5000000,Bowler,10.0,1200.0,Uncapped,Gujarat Titans
1,Shahrukh Khan,4000000,All-Rounder,7.4,888.0,PBKS,Gujarat Titans
2,Umesh Yadav,20000000,Bowler,5.8,696.0,KKR,Gujarat Titans
3,Robin Minz,2000000,Wicketkeeper,3.6,432.0,Uncapped,Gujarat Titans
4,Sushant Mishra,2000000,Bowler,2.2,264.0,Uncapped,Gujarat Titans
